In [1]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
import benchmark
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
import numpy as np

In [2]:
width = 0.1
thickness = width
cross_section = [thickness, width]

input_path = '../../data/sphere_cap_0.3.json'
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
width = 2*input_data['arm_plate_edge_offset']
thickness = width * 0.5 # 1.5 mm # FIX from mm to meters everywhere
cross_section = [thickness, width]

### Initialization

In [3]:
curr_um = umbrella_mesh.UmbrellaMesh(io)
curr_um.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))

### Parameters

In [4]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) + curr_um.rigidJointAngleDoFIndices()

### Equilibrium solve

In [5]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

#### Distance Deployment

In [6]:
rod_colors = []
for ri in range(curr_um.numSegments()):
    rod_colors.append(np.ones(curr_um.segment(ri).rod.numVertices()) * ri)

view = linkage_vis.LinkageViewer(curr_um, width=1024, height=600)
view.averagedMaterialFrames = True
view.update(scalarField = rod_colors)
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.7066666666666668, children=(PointLight(color='#999999', position=(0…

In [7]:
dof = curr_um.getDoFs()
for i in range(curr_um.numJoints()):
    if (curr_um.joint(i).jointType() == umbrella_mesh.JointType.X):
        dof[curr_um.dofOffsetForJoint(i) + 6] = 1e-3
curr_um.setDoFs(dof)

In [8]:
angles = []
gradient_norm = []
def eqm_callback(prob, i):
    angles.append(curr_um.getDoFs()[curr_um.jointAngleDoFIndices()])
    g = prob.gradient()
    fv = prob.fixedVars()
    g[fv] = 0
    for b in prob.activeBoundConstraints(prob.getVars(), g):
        g[b.idx] = 0
    gradient_norm.append(np.linalg.norm(g))
    if (i % 5 == 0):
        view.update(scalarField = rod_colors)

In [9]:
curr_um.uniformDeploymentEnergyWeight = 1
curr_um.targetDeploymentHeight = thickness * 1

In [10]:
benchmark.reset()
umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
benchmark.report()

Launching optimization with energies:
	 Full:1.22721	 Elastic:2.74224e-06	 Deployment:1.22721
0	1.22721	2.21624	2.21624	0.000209542	0
1	1.22704	2.24695	2.24695	0.00326149	0
2	1.22439	2.24069	2.24069	0.00763948	0
3	1.21989	2.22562	2.22562	0.0101839	0
4	1.21356	2.20696	2.20696	0.0113098	0
5	1.20603	2.18756	2.18756	0.0118458	0
6	1.19751	2.16869	2.16869	0.012188	0
7	1.1881	2.15071	2.15071	0.0248288	0
8	1.16784	2.14499	2.14499	0.03125	0
9	1.14431	2.14856	2.14856	0.03125	0
10	1.1203	2.1602	2.1602	0.0625	0
11	1.07124	2.4438	2.4438	0.0625	0
12	1.02725	2.65705	2.65705	0.125	0
13	0.940507	3.90519	3.90519	0.125	0
14	0.865855	4.45434	4.45434	0.25	0
15	0.727959	6.67831	6.67831	0.5	0
16	0.535903	9.91342	9.91342	1	0
17	0.297027	7.66553	7.66553	1	0
18	0.125243	4.10964	4.10964	1	0
Added constraint 352 to working set
Added constraint 379 to working set
19	0.0685212	42.4735	42.4735	1	1
20	0.0447024	6.28531	6.28531	0.345395	0
Added constraint 353 to working set
Added constraint 377 to working set
Added co

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(10,8))
plt.semilogy(gradient_norm)
plt.title('Free Gradient Components')
plt.xlabel('Newton Iteration')
plt.ylabel('Norm')
plt.grid()

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(10,8))
for i in range(len(angles[0])):
    plt.plot([a[i] / np.pi for a in angles])
plt.title('Joint Angles')
plt.xlabel('Newton Iteration')
plt.ylabel('Pi Radians')
plt.tight_layout()
plt.show()

In [ ]:
import umbrella_finite_diff
umbrella_finite_diff.gradient_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Deployment, etype=elastic_rods.EnergyType.Full)

In [ ]:
curr_um.updateSourceFrame()
umbrella_finite_diff.hessian_convergence_plot(curr_um, umbrellaEnergyType=umbrella_mesh.UmbrellaEnergyType.Full, etype=elastic_rods.EnergyType.Full)

In [ ]:
stress = curr_um.maxBendingStresses()
# stress = curr_um.twistingStresses()
view.update(scalarField=stress)